In [1]:
import os
os.chdir('C:/Users/AGB/Desktop/WeCloud_Materials/Project/Subreddit_Comments')
print(os.getcwd())

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\AGB\Desktop\WeCloud_Materials\Project\Subreddit_Comments


In [2]:
from collections import OrderedDict

# Specify subreddits of interest
subreddits = ['LateStageCapitalism','Libertarian','Anarchism','Conservative']

# Specify terms of interest
terms = ['trump','clinton','obama','russia','FBI','alt-right','republican','rnc','democrat','dnc','left','right','guns',
         'media','fake news','alternative media','mainstream media','fox','CNN','new york times','huffington','ABC','CBS',
        'antifa','anarch','libertarian','communist','jews','gay','trans','black','white','asian','southern']

# Create dataframe with summary metrics by subreddit for key terms
cons_term_df = pd.DataFrame([])

for i in subreddits:
    comment_title = i +'_processed_comments.csv'
    df = pd.read_csv(comment_title)

    perc_occurrence = []
    mean_score = []
    avg_sentiment = []
    num_positive = []
    num_negative = []
    sentiment_std = []
    avg_flesch = []
    avg_syllables = []
    avg_difficult_words = []

    for j in terms:
        term_df = df['new_comment'].str.contains(j)
        perc_occurrence.append(len(df.new_comment[term_df])/len(df['new_comment']))
        mean_score.append(df.score[term_df].mean())
        avg_sentiment.append(df.v_compound[term_df].mean())
        num_positive.append((df.v_compound[term_df]>0).sum())
        num_negative.append((df.v_compound[term_df]<0).sum())
        sentiment_std.append(df.v_compound[term_df].std())
        avg_flesch.append(df.flesch[term_df].mean())
        avg_syllables.append(df.avg_syllables[term_df].mean())
        avg_difficult_words.append(df.difficult_words[term_df].mean())

    temp = pd.DataFrame(OrderedDict({'subreddit':i,'terms':terms,'perc_occurrence':perc_occurrence,'mean_score':mean_score,'avg_sentiment':avg_sentiment,
                           'num_positive':num_positive,'num_negative':num_negative,'sentiment_std':sentiment_std,
                           'avg_flesch':avg_flesch,'avg_syllables':avg_syllables,'avg_difficult_words':avg_difficult_words}))
    cons_term_df = pd.concat([cons_term_df,temp])
    
cons_term_df = cons_term_df.reset_index(drop=True)

In [3]:
term = 'alt-right'

cons_term_df[cons_term_df['terms']==term]

,subreddit,terms,perc_occurrence,mean_score,avg_sentiment,num_positive,num_negative,sentiment_std,avg_flesch,avg_syllables,avg_difficult_words
5,LateStageCapitalism,alt-right,0.000975,14.261905,-0.041429,16,20,0.577951,8.390476,1.489286,0.229286
39,Libertarian,alt-right,0.003007,4.968992,-0.135039,104,141,0.665400,10.083721,1.631822,0.249535
73,Anarchism,alt-right,0.005504,7.016393,-0.227705,22,37,0.676489,10.095082,1.559836,0.239180
107,Conservative,alt-right,0.001300,8.629630,0.003889,25,24,0.673802,8.159259,1.505741,0.217037


In [4]:
# Fill N/As 
cons_term_df.fillna(0,inplace=True)

In [5]:
# Example of dataframe with subset of term scores in Libertarian subreddit
cons_term_df[cons_term_df['subreddit']=='Libertarian'][:10]

,subreddit,terms,perc_occurrence,mean_score,avg_sentiment,num_positive,num_negative,sentiment_std,avg_flesch,avg_syllables,avg_difficult_words
34,Libertarian,trump,0.053934,8.202291,-0.026955,2094,2202,0.628797,8.665658,1.542081,0.205336
35,Libertarian,clinton,0.005886,5.780198,-0.047287,227,244,0.669380,8.867921,1.518040,0.222990
36,Libertarian,obama,0.017333,4.503026,-0.072253,638,721,0.654839,8.615131,1.534055,0.228527
37,Libertarian,russia,0.008288,3.257384,-0.184543,239,412,0.624838,9.097750,1.572996,0.237595
38,Libertarian,FBI,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000
39,Libertarian,alt-right,0.003007,4.968992,-0.135039,104,141,0.665400,10.083721,1.631822,0.249535
40,Libertarian,republican,0.024234,8.240019,0.091582,1102,826,0.614984,9.583405,1.617484,0.231501
41,Libertarian,rnc,0.000210,3.611111,0.046111,7,5,0.655413,58.172222,5.823333,0.276667
42,Libertarian,democrat,0.024070,5.062470,0.058015,1056,875,0.653788,8.933366,1.542877,0.219743
43,Libertarian,dnc,0.001492,2.757812,-0.213359,37,82,0.622316,8.179688,1.510312,0.202812


In [6]:
table =  pd.pivot_table(cons_term_df, values='avg_sentiment', index=['terms'], columns=['subreddit'])

In [7]:
# Create a correlation matrix between subreddits and average sentiment score on given topics to find similarities in perspective
table.corr()

subreddit,Anarchism,Conservative,LateStageCapitalism,Libertarian
subreddit,,,,
Anarchism,1.000000,0.366489,0.595983,0.391347
Conservative,0.366489,1.000000,0.537755,0.708094
LateStageCapitalism,0.595983,0.537755,1.000000,0.479355
Libertarian,0.391347,0.708094,0.479355,1.000000


In [8]:
df.to_csv('!term_comparison_summary.csv',index=False)

In [54]:
# Manually inspect comments containing a given term for a given subreddit to investigate sample
subreddit = 'Libertarian'
contains = 'anarch'

comment_title = subreddit +'_processed_comments.csv'
df = pd.read_csv(comment_title)

pd.set_option('display.max_colwidth', -1)
df[df['new_comment'].str.contains(contains)][['v_compound','new_comment']]

,v_compound,new_comment
6,-0.84,"&gt;no one likes it when commies do that either. apparently you do, which is why you were defending a """"""nazi"""""" (who's actually an anarcho-capitalist) being prosecuted for things he said on his radio show. &gt;but this was about one cesspool dwelling nazi who came out fighting, and was in posession of pepper spray which he allegedly used. in self-defense, you sick fuck."
31,-0.05,"depends on a definition of the terms. anarchists would argue that liberty and capitalism are contradictory, but here we are. if you remember, 'libertarian' used to be synonymous with (left-wing) anarchism. you guys changed the meaning of the term, not us."
81,-0.48,"well i'm not sure what you're worried about then, since antifas are, by the definition you've chosen, not communists. i've certainly never met an anarchist who wanted another ussr, stalin slaughtered anarchists en masse."
95,0.19,"lsc is crawling with tankies, yes. i'm not sure what that has to do with anarchism and antifa, though. have you seen any anarchists calling for another ussr?"
116,0.44,might get better answers at /r/anarcho_capitalism/
238,0.62,communism is great for the same reasons anarchy is: both depend on human beings acting differently than humans actually act.
245,-0.68,"no, it means that the systems in russia and china are not how marx described them. in marxist terms communism is an end state after a country/the world has gone through a socialist phase. communism is a pipe dream, like anarchy it depends on people acting differently from real people. stalinism and maoism were bad in part because central planning doesn't have useful information flows and in a very large part that both were run by homicidal paranoid psychopaths."
270,0.92,"but antifa are not stormtroppers (or, more propertly sa, sturmabteilung). stormtroopers are what we call special forces today: highly trained and armed. the sa was the paramilitary arm of the nazi party, sent out to beat up political opponents. antifa are anarchists who think they can set the world on fire and so build something better in the ashes. ideologically they are closer in some ways to libertarians than they are to democrats. now i want to be clear about this. i think that in many ways libertarianism is ideologically incoherent. that is, there are dozens of different views that are under the umbrella libertarian. one of them is the nap non-violent side. another is the anarchist no hierarchy tear it down side. these should not be considered one group but you are all such small groups to split more leaves nothing left."
405,0.00,anarchy and communism
452,-0.33,"martin luther king jr. advocated a superior society, not an anarchist society. do you want corporations to tread on you? i don't. do you want people to go bankrupt because of medical debt? i don't. do you want to deal with corrupt local police who serve the employer? i don't. do you want gangs that operate unopposed? i don't."
